### 1. Prepare the working environment

In [ ]:

using Pkg
Pkg.add("IceFloeTracker")
Pkg.add("DataFrames")

HOME = "../../.."; # path to the root of the project two levels up

In [ ]:
using IceFloeTracker
using DataFrames
using IceFloeTracker: deserialize, addfloemasks!, adduuid!, add_passtimes!, addψs!, long_tracker, addlatlon!, imshow

### 2. Set thresholds for the algorithm

In [ ]:
# Set thresholds
condition_thresholds = IceFloeTracker.condition_thresholds
mc_thresholds = IceFloeTracker.mc_thresholds
@show condition_thresholds
@show mc_thresholds

### 3. Load data

In [ ]:
begin # Load data
    pth = joinpath(HOME, "test", "test_inputs", "tracker")
    floedata = deserialize(joinpath(pth, "tracker_test_data.dat"))
    # test/test_inputs/tracker/tracker_test_data.dat
    passtimes = deserialize(joinpath(pth, "passtimes.dat"))
    props, imgs = deepcopy.([floedata.props, floedata.imgs])

    # Add required fields to props
    addfloemasks!(props, imgs)
    add_passtimes!(props, passtimes)
    addψs!(props)
    adduuid!(props)
end

# Delete some floes
deleteat!(props[1], 1); # delete the first floe in day 1 so it doesn't have a match in day 2
deleteat!(props[2], 5); # delete the fifth floe in day 2 so it doesn't have a match in day 1

# All floes in days 1 and 2 have a match in day 3
# Expected: 5 trajectories, 3 of length 3 and 2 of length 2
nothing # suppress output -- not particularly informative. See the next block.

### 4. View floe data

In [ ]:
# Day 1
props[1][!, [:uuid, :passtime, :area]]

In [ ]:
imshow(imgs[1])

In [ ]:
imshow(props[1][1, :mask])

In [ ]:
# Day 2
props[2][!, [:uuid, :passtime, :area]]

In [ ]:
imshow(imgs[2]) # slightly rotated version of the image in day 1

In [ ]:
imshow(props[2][1, :mask])

In [ ]:
# Day 3
props[3][!, [:uuid, :passtime, :area]]

In [ ]:
imshow(props[3][1, :mask]) # missing in day 1

In [ ]:
imshow(props[3][5, :mask]) # missing in day 2

### 5. Pair and label floes

In [ ]:
# Get paired floes with labels
trajectories = long_tracker(props, condition_thresholds, mc_thresholds);

### 6. View trajectories and _goodness_ of pairings

In [ ]:
trajectories[!, [:ID, :passtime, :area_mismatch, :corr]]

7. Add latitude/longitude data to trajectories

In [ ]:
latlonimgpth = joinpath(HOME, "test", "test_inputs", "beaufort-chukchi-seas_truecolor.2020162.aqua.250m.tiff")
addlatlon!(trajectories, latlonimgpth)

In [ ]:
# View trajectories with lat/lon data
# Note that the areas are now in physical units (km^2)
cols = [:ID, :area, :passtime, :latitude, :longitude]
trajectories[!, cols]

In [ ]:
grouped_trajectories = groupby(trajectories, :ID);

In [ ]:
imshow.(grouped_trajectories[1].mask)

In [ ]:
imshow.(grouped_trajectories[2].mask)

In [ ]:
imshow.(grouped_trajectories[3].mask)

In [ ]:
imshow.(grouped_trajectories[4].mask)

In [ ]:
imshow.(grouped_trajectories[5].mask)